# TensorFlow Estimators

Estimators and datasets make up TensorFlow high-level APIs. 

![](assets/tensorflow_programming_environment.png)

## Estimator program structure

- Import and parse the data
- Create feature columns to describe the data
- Select model
- Train model
- Evaluate model
- Make predictions from model

![](assets/tensorflow-estimator.png)

[Premade Estimators](https://www.tensorflow.org/get_started/premade_estimators "" target="_blank")

In [1]:
from __future__ import print_function
import os
import pandas as pd
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


n.b. Below we will use the common root path for models as /tmp/iris. As such, tensorboard can be run as `tensorboard --logdir=/tmp/iris` and we can compare all our results next to each other

## Data

Load the data into a pandas data frame

We'll start with the [UCI Machine Learning: Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris). It has four numerical features classifying three types of iris plant. Keeping this simple means we can focus on the infrastructure around estimators without getting complexity introduced by the data.

In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

We'll grab the data and store it locally and then load it up into a pandas data frame. With other data, you'd look at missing values, distributions etc but this data is very clean and we want to focus on the estimator methods. 

In [3]:
def get_data(remote_url):
    CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
    
    local_path = tf.keras.utils.get_file(os.path.basename(remote_url), remote_url, cache_dir=os.getcwd())
    
    df = pd.read_csv(local_path, header=0, names=CSV_COLUMN_NAMES)
    
    features, label = df, df.pop('Species')
    
    return features, label

In [4]:
train_features, train_labels = get_data(TRAIN_URL)
test_features, test_labels = get_data(TEST_URL)

In [5]:
N_CLASSES = len(train_labels.unique())

In [7]:
train_features.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [8]:
train_labels.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

Estimators use an input function that is called to return data. It is a `callable` that returns a tuple - dictionary of features to tensors and label as tensor. The estimator module has a couple of [built in utilities](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs) for some common cases such as numpy and pandas data sources. For a much larger case of dataset capabilities, look at the [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) module.

In [59]:
def make_input_fn(feature_df, label_df, shuffle=False, batch_size=128, num_epochs=None):
    input_fn = tf.estimator.inputs.pandas_input_fn(
        feature_df, y=label_df, batch_size=batch_size, 
        shuffle=shuffle, num_epochs=num_epochs, target_column='Species')
    
    return input_fn

[Feature Columns](https://www.tensorflow.org/get_started/feature_columns) act as the intermediaries between the raw data and estimators. Here we are just using numerical columns (number in, number out). 

Other types of columns include:
- Bucketized: split value into different categories based on numerical range (e.g. age buckets -> [0-18, 18-25, 25-35, 35-65, 65+])
- Categorical: one-hot encoding
- Hashed: hash buckets for computing categories
- Crossed: combination of features
- Embedding: categorical features to dense (e.g. word vocabulary, post codes etc)

In [10]:
feature_columns = []
for key in train_features.keys():
    feature_columns.append(tf.feature_column.numeric_column(key))

## Model

Make sure we save summary values often for our demo. Normally you would use much larger values so your compute can concentrate on compute rather than writing to disk.

In [60]:
run_config = tf.estimator.RunConfig(save_summary_steps=10)

### Linear Model

Start with [linear classifier](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearClassifier "" target="_blank")

In [61]:
def make_linear_classifier(model_dir):
    classifier = tf.estimator.LinearClassifier(
        feature_columns=feature_columns, 
        n_classes=N_CLASSES,
        model_dir=model_dir, 
        config=run_config
    )
    return classifier

In [62]:
linear_classifier = make_linear_classifier('/tmp/iris/linear')

INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris/linear', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10b866470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
linear_classifier.train(input_fn=make_input_fn(train_features, train_labels, shuffle=True), max_steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris/linear/model.ckpt.
INFO:tensorflow:loss = 140.62234, step = 1
INFO:tensorflow:global_step/sec: 256.064
INFO:tensorflow:loss = 39.921, step = 101 (0.395 sec)
INFO:tensorflow:global_step/sec: 534.62
INFO:tensorflow:loss = 27.999634, step = 201 (0.184 sec)
INFO:tensorflow:global_step/sec: 595.149
INFO:tensorflow:loss = 25.861763, step = 301 (0.168 sec)
INFO:tensorflow:global_step/sec: 596.933
INFO:tensorflow:loss = 23.64569, step = 401 (0.168 sec)
INFO:tensorflow:global_step/sec: 590.322
INFO:tensorflow:loss = 17.986433, step = 501 (0.169 sec)
INFO:tensorflow:global_step/sec: 525.1
INFO:tensorflow:loss = 16.413773, step = 601 (0.192 sec)
INFO:tensorflow:global_step/sec: 592.522
INFO:tensorflow:loss = 

In [65]:
linear_classifier.evaluate(input_fn=make_input_fn(test_features, test_labels, batch_size=1), steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-07-02:05:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris/linear/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-05-07-02:05:14
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96, average_loss = 0.13049284, global_step = 1000, loss = 0.13049284


{'accuracy': 0.96,
 'average_loss': 0.13049284,
 'loss': 0.13049284,
 'global_step': 1000}

### NN Model

Make it a [neural network](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier "" target="_blank"). All we need to do to move from a linear model to a deep neural network is change the class name and provide an extra parameter defining the size of the hidden layers in the model.

In [66]:
def make_dnn_classifier(model_dir, hidden_units=[10, 10]):
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns, 
        hidden_units=hidden_units, 
        n_classes=N_CLASSES,
        model_dir=model_dir, 
        config=run_config
    )
    return classifier

In [67]:
classifier = make_dnn_classifier('/tmp/iris/dnn')

INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris/dnn', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10a53f470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Train the model

In [68]:
classifier.train(input_fn=make_input_fn(train_features, train_labels, shuffle=True), max_steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris/dnn/model.ckpt.
INFO:tensorflow:loss = 446.35168, step = 1
INFO:tensorflow:global_step/sec: 272.921
INFO:tensorflow:loss = 18.026764, step = 101 (0.369 sec)
INFO:tensorflow:global_step/sec: 589.557
INFO:tensorflow:loss = 10.664019, step = 201 (0.169 sec)
INFO:tensorflow:global_step/sec: 593.778
INFO:tensorflow:loss = 11.549435, step = 301 (0.168 sec)
INFO:tensorflow:global_step/sec: 586.865
INFO:tensorflow:loss = 8.187953, step = 401 (0.170 sec)
INFO:tensorflow:global_step/sec: 550.81
INFO:tensorflow:loss = 7.4220843, step = 501 (0.182 sec)
INFO:tensorflow:global_step/sec: 558.677
INFO:tensorflow:loss = 8.770698, step = 601 (0.179 sec)
INFO:tensorflow:global_step/sec: 602.715
INFO:tensorflow:loss =

In [69]:
classifier.evaluate(input_fn=make_input_fn(test_features, test_labels, batch_size=1), steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-07-02:05:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris/dnn/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-05-07-02:05:36
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.93, average_loss = 0.068787344, global_step = 1000, loss = 0.068787344


{'accuracy': 0.93,
 'average_loss': 0.068787344,
 'loss': 0.068787344,
 'global_step': 1000}

In [71]:
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}
predict_df = pd.DataFrame.from_dict(predict_x)
predictions = classifier.predict(input_fn=make_input_fn(predict_df, label_df=None, shuffle=False, batch_size=1, num_epochs=1))

predictions = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris/dnn/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [72]:
predictions

[{'logits': array([ 16.399452 ,  10.220468 , -11.1659565], dtype=float32),
  'probabilities': array([9.9793172e-01, 2.0682446e-03, 1.0656042e-12], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-2.015658  ,  6.1454844 , -0.20347175], dtype=float32),
  'probabilities': array([2.8495639e-04, 9.9797004e-01, 1.7450220e-03], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-8.667519 ,  1.99388  ,  6.1452847], dtype=float32),
  'probabilities': array([3.63159643e-07, 1.54983075e-02, 9.84501302e-01], dtype=float32),
  'class_ids': array([2]),
  'classes': array([b'2'], dtype=object)}]

## Automated train and evaluate

[Train and evaluate](https://www.tensorflow.org/api_docs/python/tf/estimator/train_and_evaluate "" target="_blank") is a utility function to provide consistent behaviour for local and distributed configurations. Currently just supports between-graph replication (graph per device).

In [73]:
estimator = make_dnn_classifier('/tmp/iris/train_and_evaluate')

train_input_fn = make_input_fn(train_features, train_labels, shuffle=True, num_epochs=None)
train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=100)

test_input_fn = make_input_fn(test_features, test_labels, batch_size=1, num_epochs=None)
test_spec = tf.estimator.EvalSpec(test_input_fn)

tf.estimator.train_and_evaluate(estimator, train_spec, test_spec)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris/train_and_evaluate', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10b663208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INF

In [ ]:
feature_spec = {
    'SepalLength': tf.FixedLenFeature([], dtype=tf.float32),
    'SepalWidth': tf.FixedLenFeature([], dtype=tf.float32),
    'PetalLength': tf.FixedLenFeature([], dtype=tf.float32),
    'PetalWidth': tf.FixedLenFeature([], dtype=tf.float32)
}
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

estimator.export_savedmodel('iris-exports', serving_input_receiver_fn)

## Bonus: data input pipeline and performance

There is a lot of guidance around optimising the performance of your input pipeline. This was the reason `tf.data` was made and there is a complete guide at [Input Pipeline Performance Guide](https://www.tensorflow.org/versions/master/performance/datasets_performance). 

Considering the popular use of CSV datasets, recently a contrib function has been added to combine all of this guidance into a single method call. See below:

In [ ]:
contrib_dataset = tf.contrib.data.make_csv_dataset(train_path, batch_size=128, column_names=CSV_COLUMN_NAMES, label_name='Species', header=False)